## import packages

In [1]:
import sys

sys.path.append('../lib/')
sys.path.append('../lib/net/')
sys.path.append('../lib/rpn/')
sys.path.append('../lib/utils/iou3d/')
sys.path.append('../lib/utils/roipool3d/')
sys.path.append('../lib/utils/roipool3d/build/')
sys.path.append('../pointnet2_lib/')
sys.path.append('../pointnet2_lib/')
sys.path.append('../pointnet2_lib/pointnet2/')
sys.path.append('../pointnet2_lib/tools/')
sys.path.append('./train_utils/')
sys.path.append('./kitti_object_eval_python/')
sys.path.append('./cfgs/')

import _init_path
import os
import numpy as np
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
from lib.net.point_rcnn import PointRCNN
from lib.datasets.kitti_rcnn_dataset import KittiRCNNDataset
import tools.train_utils.train_utils as train_utils
from lib.utils.bbox_transform import decode_bbox_target
from tools.kitti_object_eval_python.evaluate import evaluate as kitti_evaluate

from lib.config import cfg, cfg_from_file, save_config_to_file, cfg_from_list
import argparse
import lib.utils.kitti_utils as kitti_utils
import lib.utils.iou3d.iou3d_utils as iou3d_utils
from datetime import datetime
import logging
import re
import glob
import time
from tensorboardX import SummaryWriter
import tqdm

import pandas as pd

In [2]:
import numpy as np
import pandas as pd
import os
import gc


import json
import math 
import sys
from datetime import datetime
import time
from typing import Tuple,List

import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import sklearn.metrics
from PIL import Image

from matplotlib.axes import Axes
from matplotlib import animation, rc
import plotly.graph_objs as go
import plotly.tools as ts
from plotly.offline import plot, init_notebook_mode
import plotly.figure_factory as ft
init_notebook_mode(connected=True)
from pyquaternion import Quaternion
import seaborn as sns
import warnings

from lyft_dataset_sdk.utils.map_mask import MapMask
from lyft_dataset_sdk.lyftdataset import LyftDataset
from lyft_dataset_sdk.utils.geometry_utils import view_points, box_in_image, BoxVisibility
from lyft_dataset_sdk.utils.geometry_utils import view_points, transform_matrix
from pathlib import Path

from lyft_dataset_sdk.lyftdataset import LyftDataset,LyftDatasetExplorer
from lyft_dataset_sdk.utils.data_classes import LidarPointCloud, Box, Quaternion
from lyft_dataset_sdk.utils.geometry_utils import view_points, transform_matrix
import time
from lyft_dataset_sdk.utils.map_mask import MapMask

import struct
from abc import ABC, abstractmethod
from functools import reduce
from typing import Tuple, List, Dict
import copy
from shutil import copyfile

from PIL import Image

## args input

In [3]:
np.random.seed(1024)  # set the same seed

from easydict import EasyDict as edict

args = edict()

In [4]:
saved_class = 'car'

args.ckpt = '../trainedModel/car/rcnn/ckpt/rpn30/checkpoint_epoch_2.pth'

args.output_dir = '../predicted/mirror'

In [5]:
args.cfg_file = 'cfgs/' + saved_class + '.yaml'
args.eval_mode = 'rcnn'


args.eval_all = False
args.test = True 

args.rpn_ckpt = None
args.rcnn_ckpt = None


args.batch_size = 4
args.workers = 4
args.extra_tag = 'default'
args.ckpt_dir = None


args.save_result = False
args.save_rpn_feature = False


args.random_select = True
args.start_epoch = 0
args.rcnn_eval_roi_dir = None
args.rcnn_eval_feature_dir = None
args.set_cfgs = None


## functions

### ling's

#### create_logger

In [6]:
def create_logger(log_file):
    log_format = '%(asctime)s  %(levelname)5s  %(message)s'
    logging.basicConfig(level=logging.INFO, format=log_format, filename=log_file)
    console = logging.StreamHandler()
    console.setLevel(logging.INFO)
    console.setFormatter(logging.Formatter(log_format))
    logging.getLogger(__name__).addHandler(console)
    return logging.getLogger(__name__)

#### create_dataloader

In [7]:
def create_dataloader(logger):
    mode = 'TEST' 
    DATA_PATH = os.path.join('..', 'data')

    # create dataloader
    test_set = KittiRCNNDataset(root_dir=DATA_PATH, npoints=cfg.RPN.NUM_POINTS, split='test', mode=mode,
                                random_select=args.random_select,
                                rcnn_eval_roi_dir=args.rcnn_eval_roi_dir,
                                rcnn_eval_feature_dir=args.rcnn_eval_feature_dir,
                                classes=cfg.CLASSES,
                                logger=logger)

    test_loader = DataLoader(test_set, batch_size=args.batch_size, shuffle=False, pin_memory=True,
                             num_workers=args.workers, collate_fn=test_set.collate_batch)

    return test_loader

#### save_predicted_boxes3d

In [8]:
def save_predicted_boxes3d(pred_boxes3d, scores_pred, savefile):
    """
        args:
            pred_boxes3d: numpy.ndarray (ncars * 7[x, y, z, height, width, length, yaw])
            
            scores_pred: numpy.ndarray (ncars * 1)
            
            savefile: savefile name, save format: [confidence, x, y, z, width, length, height, yaw, classname]
    """
    
    
    with open(savefile, 'w') as f:
        for k in range(pred_boxes3d.shape[0]):
            #print('%.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %s' %
            #      (scores_pred[k], pred_boxes3d[k, 0], pred_boxes3d[k, 1], pred_boxes3d[k, 2], 
            #       pred_boxes3d[k, 4], pred_boxes3d[k, 5], pred_boxes3d[k, 3],pred_boxes3d[k, 6],
            #      saved_class), file=f)
            print('%.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %s' %
                  (scores_pred[k], pred_boxes3d[k, 0], pred_boxes3d[k, 1], -pred_boxes3d[k, 2], 
                   pred_boxes3d[k, 4], pred_boxes3d[k, 5], pred_boxes3d[k, 3],3.142-pred_boxes3d[k, 6],
                  saved_class), file=f)
    

#### load_ckpt_based_on_args

In [9]:
def load_ckpt_based_on_args(model, logger):
    if args.ckpt is not None:
        train_utils.load_checkpoint(model, filename=args.ckpt, logger=logger)

#### detect_one_epoch_join()

In [10]:
def detect_one_epoch_joint(model, dataloader, epoch_id, result_dir, logger, sample_token):
    '''
    use joint (rpn + rcnn) model to detect object
    
    output:
        predicted 3dboxes stored in result_dir
    
    '''
    np.random.seed(666)
    MEAN_SIZE = torch.from_numpy(cfg.CLS_MEAN_SIZE[0]).cuda()

    predicted_boxes3d_dir = os.path.join(result_dir, 'pred_boxes3d', cfg.CLASSES)
    os.makedirs(predicted_boxes3d_dir, exist_ok=True)


    logger.info('---- EPOCH %s JOINT PREDICTION ----' % epoch_id)
    logger.info('==> Output file: %s' % predicted_boxes3d_dir)
    model.eval()

    thresh_list = [0.1, 0.3, 0.5, 0.7, 0.9]
    total_recalled_bbox_list, total_gt_bbox = [0] * 5, 0
    total_roi_recalled_bbox_list = [0] * 5
    dataset = dataloader.dataset
    cnt = final_total = total_cls_acc = total_cls_acc_refined = total_rpn_iou = 0

    progress_bar = tqdm.tqdm(total=len(dataloader), leave=True, desc='eval')
    for data in dataloader:
        cnt += 1
        sample_id, pts_rect, pts_features, pts_input = \
            data['sample_id'], data['pts_rect'], data['pts_features'], data['pts_input']
        batch_size = len(sample_id)
        inputs = torch.from_numpy(pts_input).cuda(non_blocking=True).float()
        input_data = {'pts_input': inputs}

        # model inference
        ret_dict = model(input_data)

        roi_scores_raw = ret_dict['roi_scores_raw']  # (B, M)
        roi_boxes3d = ret_dict['rois']  # (B, M, 7)
        seg_result = ret_dict['seg_result'].long()  # (B, N)

        rcnn_cls = ret_dict['rcnn_cls'].view(batch_size, -1, ret_dict['rcnn_cls'].shape[1])
        rcnn_reg = ret_dict['rcnn_reg'].view(batch_size, -1, ret_dict['rcnn_reg'].shape[1])  # (B, M, C)

        # bounding box regression
        anchor_size = MEAN_SIZE
        if cfg.RCNN.SIZE_RES_ON_ROI:
            assert False

        pred_boxes3d = decode_bbox_target(roi_boxes3d.view(-1, 7), rcnn_reg.view(-1, rcnn_reg.shape[-1]),
                                          anchor_size=anchor_size,
                                          loc_scope=cfg.RCNN.LOC_SCOPE,
                                          loc_bin_size=cfg.RCNN.LOC_BIN_SIZE,
                                          num_head_bin=cfg.RCNN.NUM_HEAD_BIN,
                                          get_xz_fine=True, get_y_by_bin=cfg.RCNN.LOC_Y_BY_BIN,
                                          loc_y_scope=cfg.RCNN.LOC_Y_SCOPE, loc_y_bin_size=cfg.RCNN.LOC_Y_BIN_SIZE,
                                          get_ry_fine=True).view(batch_size, -1, 7)
        
        print(type(pred_boxes3d))

        # scoring
        if rcnn_cls.shape[2] == 1:
            raw_scores = rcnn_cls  # (B, M, 1)

            norm_scores = torch.sigmoid(raw_scores)
            pred_classes = (norm_scores > cfg.RCNN.SCORE_THRESH).long()
        else:
            pred_classes = torch.argmax(rcnn_cls, dim=1).view(-1)
            cls_norm_scores = F.softmax(rcnn_cls, dim=1)
            raw_scores = rcnn_cls[:, pred_classes]
            norm_scores = cls_norm_scores[:, pred_classes]
            
        
        disp_dict = {'mode': 'Predicting'}
        progress_bar.set_postfix(disp_dict)
        progress_bar.update()

        # scores thresh
        inds = norm_scores > cfg.RCNN.SCORE_THRESH

        for k in range(batch_size):
            cur_inds = inds[k].view(-1)
            if cur_inds.sum() == 0:
                continue

            pred_boxes3d_selected = pred_boxes3d[k, cur_inds]
            raw_scores_selected = raw_scores[k, cur_inds]
            norm_scores_selected = norm_scores[k, cur_inds]
            

            # NMS thresh
            # rotated nms
            boxes_bev_selected = kitti_utils.boxes3d_to_bev_torch(pred_boxes3d_selected)
            keep_idx = iou3d_utils.nms_gpu(boxes_bev_selected, raw_scores_selected, cfg.RCNN.NMS_THRESH).view(-1)
            pred_boxes3d_selected = pred_boxes3d_selected[keep_idx]
            scores_selected = raw_scores_selected[keep_idx]
            pred_boxes3d_selected, scores_selected = pred_boxes3d_selected.cpu().numpy(), scores_selected.cpu().numpy()

            cur_sample_id = sample_id[k]
            savefile = os.path.join(predicted_boxes3d_dir, '%06d.txt' % cur_sample_id)
            
            save_predicted_boxes3d(pred_boxes3d_selected, norm_scores_selected, savefile)

### Tong's

#### link folders

In [11]:
!ln -s ../../3d-object-detection-for-autonomous-vehicles/test_images images
!ln -s ../../3d-object-detection-for-autonomous-vehicles/test_maps maps
!ln -s ../../3d-object-detection-for-autonomous-vehicles/test_lidar lidar

ln: failed to create symbolic link 'images/test_images': File exists
ln: failed to create symbolic link 'maps/test_maps': File exists
ln: failed to create symbolic link 'lidar/test_lidar': File exists


#### load test dataset

In [12]:
test_dataset = LyftDataset(data_path='.', json_path='../../3d-object-detection-for-autonomous-vehicles/test_data', verbose=True)

JSON file instance.json missing, using empty list
JSON file sample_annotation.json missing, using empty list
9 category,
17 attribute,
4 visibility,
0 instance,
8 sensor,
168 calibrated_sensor,
219744 ego_pose,
218 log,
218 scene,
27468 sample,
219744 sample_data,
0 sample_annotation,
1 map,
Done loading in 1.9 seconds.
Reverse indexing ...
Done reverse indexing in 0.7 seconds.


#### Output transformation: global_from_rect

In [13]:
def label_global_from_rect(sample_ID,args):
    #args = [x,y,z,width,length,height,yaw]
    # get the label from the sample_ID
    import math
    sample_token=sample_ID 
    sample = test_dataset.get("sample", sample_token)

    sample_lidar_token = sample["data"]["LIDAR_TOP"]
    lidar_data = test_dataset.get("sample_data", sample_lidar_token)

    # get lidar calibration info
    ego_pose = test_dataset.get("ego_pose", lidar_data["ego_pose_token"])
    calibrated_sensor = test_dataset.get("calibrated_sensor", lidar_data["calibrated_sensor_token"])

    # get the transformation matrix from calibration info
    Tr_rect_sensor = np.array([[-1,0,0,0],[0,0,1,0],[0,-1,0,0],[0,0,0,1]])

    global_from_car = transform_matrix(ego_pose['translation'],
                                       Quaternion(ego_pose['rotation']), inverse=False)
    car_from_sensor = transform_matrix(calibrated_sensor['translation'], Quaternion(calibrated_sensor['rotation']),
                                        inverse=False)
    global_from_sensor = np.dot(global_from_car,car_from_sensor)
    
    global_from_rect = np.dot(global_from_sensor,Tr_rect_sensor)
    
    center_x = args[0]
    center_y = args[1]
    center_z = args[2]
    width = args[3]
    length = args[4]
    height = args[5]
    yaw = args[6]
        
        
        
    # transfrom the center point from global to sensor (center_x,center_y,center_z)->(x,y,z)
    tmp = np.array([[center_x],[center_y],[center_z],[1.0]])
    coor = np.dot(global_from_rect,tmp)
    x = coor[0][0]
    y = coor[1][0]
    z = coor[2][0]

    # transform the yaw from sensor to global yaw->yaw_sensor
    # yaw vector += center vector
    x_yaw = center_x - np.sin(yaw)
    z_yaw = center_z + np.cos(yaw)
    y_yaw = center_y
    # transfer yaw vector to sensor
    coor_yaw = np.dot(global_from_rect,np.array([[x_yaw],[y_yaw],[z_yaw],[1.0]]))
    x_yaw_global = coor_yaw[0][0]
    y_yaw_global = coor_yaw[1][0]
    z_yaw_global = coor_yaw[2][0]
    # the angle between yaw_sensor and the y axis
    #yaw_new = np.arccos(np.clip(np.dot((1,0),(y_yaw_global - y,x_yaw_global - x)), -1.0, 1.0))
    yaw_new = math.atan2(-(x_yaw_global - x),y_yaw_global - y)
    
        
    return [x,y,z,width,length,height,yaw_new]

#### lidar_filepath_from_id

In [14]:
def lidar_filepath_from_id(sample_token):
    sample = test_dataset.get("sample", sample_token)

    sample_lidar_token = sample["data"]["LIDAR_TOP"]
    
    lidar_path = test_dataset.get_sample_data_path(sample_lidar_token)
    return lidar_path

#### write_image_from_id

In [15]:
def write_image_from_id(image_folder,kitti_id,sample_token):
    

    sample = test_dataset.get("sample", sample_token)

    sample_cam_token = sample["data"]["CAM_FRONT_LEFT"]
    cam_data = test_dataset.get("sample_data", sample_cam_token)
    cam_filepath = test_dataset.get_sample_data_path(sample_cam_token)
    
    im = Image.open(cam_filepath)
    im.save(os.path.join(image_folder,"{:06n}".format(kitti_id)+".png"), "PNG")

#### calib file

In [16]:
def write_calib(calib_folder,idx):
    
    label_save_fn = os.path.join(calib_folder,"{:06n}".format(idx)+'.txt')
    Tr_lyft_rect = np.array([-1,  0,  0,  0, 0,  0, -1,  0, 0,  1,  0,  0])
    R0_rect = np.array([1,0,0,0,1,0,0,0,1])
    with open(label_save_fn, 'w') as f:

        print('P0: %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f'  %
                          (1,0,0,0,0,1,0,0,0,0,1,0),file=f)
        print('P1: %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f'  %
                          (1,0,0,0,0,1,0,0,0,0,1,0),file=f)
        print('P2: %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f'  %
                          (1,0,0,0,0,1,0,0,0,0,1,0),file=f)
        print('P3: %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f'  %
                          (1,0,0,0,0,1,0,0,0,0,1,0),file=f)

        print('R0_rect: %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f' %
                          (R0_rect[0], R0_rect[1],R0_rect[2], R0_rect[3], R0_rect[4],
                           R0_rect[5], R0_rect[6], R0_rect[7], R0_rect[8]),file=f)

        #print('Tr_velo_to_cam: %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f' %
        #                  (-1,0,0,0,0,0,-1,0,0,1,0,0),file=f)
        # z= -z
        print('Tr_velo_to_cam: %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f' %
                          (-1,0,0,0,0,0,-1,0,0,-1,0,0),file=f)

## main

In [17]:
# merge config and log to file
if args.cfg_file is not None:
    cfg_from_file(args.cfg_file)
       
cfg.TAG = os.path.splitext(os.path.basename(args.cfg_file))[0]


# args.eval_mode == 'rcnn':
cfg.RCNN.ENABLED = True
cfg.RPN.ENABLED = cfg.RPN.FIXED = True
root_result_dir = os.path.join('../', 'output', 'rcnn', cfg.TAG)
ckpt_dir = args.ckpt_dir

if args.output_dir is not None:
    root_result_dir = args.output_dir
    
 # set epoch_id and output dir
num_list = re.findall(r'\d+', args.ckpt) if args.ckpt is not None else []
epoch_id = num_list[-1] if num_list.__len__() > 0 else 'no_number'
root_result_dir = os.path.join(root_result_dir, 'epoch_%s' % epoch_id)

os.makedirs(root_result_dir, exist_ok=True)

log_file = os.path.join(root_result_dir, 'log_eval_one.txt')
logger = create_logger(log_file)
logger.info('**********************Start logging**********************')
for key, val in vars(args).items():
    logger.info("{:16} {}".format(key, val))
save_config_to_file(cfg, logger=logger)

/home/lingling/Downloads/Lyft_PointRCNN/tools/../lib/config.py:187: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.

2019-11-12 16:50:02,927   INFO  **********************Start logging**********************
2019-11-12 16:50:02,928   INFO  ckpt             ../trainedModel/car/rcnn/ckpt/rpn30/checkpoint_epoch_2.pth
2019-11-12 16:50:02,928   INFO  output_dir       ../predicted/mirror
2019-11-12 16:50:02,929   INFO  cfg_file         cfgs/car.yaml
2019-11-12 16:50:02,929   INFO  eval_mode        rcnn
2019-11-12 16:50:02,929   INFO  eval_all         False
2019-11-12 16:50:02,930   INFO  test             True
2019-11-12 16:50:02,930   INFO  rpn_ckpt         None
2019-11-12 16:50:02,931   INFO  rcnn_ckpt        None
2019-11-12 16:50:02,931   INFO  batch_size       4
2019-11-12 16:50:02,931   INFO  workers          4
2019-11-12 16:50:02,931   INFO  extra_tag        default
2019-11-

2019-11-12 16:50:02,968   INFO  cfg.TRAIN.GRAD_NORM_CLIP: 1.0
2019-11-12 16:50:02,968   INFO  cfg.TRAIN.RPN_PRE_NMS_TOP_N: 9000
2019-11-12 16:50:02,968   INFO  cfg.TRAIN.RPN_POST_NMS_TOP_N: 512
2019-11-12 16:50:02,969   INFO  cfg.TRAIN.RPN_NMS_THRESH: 0.85
2019-11-12 16:50:02,969   INFO  cfg.TRAIN.RPN_DISTANCE_BASED_PROPOSE: True
2019-11-12 16:50:02,969   INFO  
cfg.TEST = edict()
2019-11-12 16:50:02,969   INFO  cfg.TEST.SPLIT: val
2019-11-12 16:50:02,970   INFO  cfg.TEST.RPN_PRE_NMS_TOP_N: 9000
2019-11-12 16:50:02,970   INFO  cfg.TEST.RPN_POST_NMS_TOP_N: 100
2019-11-12 16:50:02,970   INFO  cfg.TEST.RPN_NMS_THRESH: 0.8
2019-11-12 16:50:02,970   INFO  cfg.TEST.RPN_DISTANCE_BASED_PROPOSE: True


In [18]:
calib_folder = "/home/lingling/Downloads/Lyft_PointRCNN/data/KITTI/object/testing/calib/"
lidar_folder = '/home/lingling/Downloads/Lyft_PointRCNN/data/KITTI/object/testing/velodyne'
image_folder = '/home/lingling/Downloads/Lyft_PointRCNN/data/KITTI/object/testing/image_2'
testtxtfile = '/home/lingling/Downloads/Lyft_PointRCNN/data/KITTI/ImageSets/test.txt'

os.makedirs(calib_folder, exist_ok=True)
os.makedirs(lidar_folder, exist_ok=True)
os.makedirs(image_folder, exist_ok=True)


#os.makedirs(lidar_folder)

df = pd.read_csv('sample_submission.csv')
lyfttest_Id = df['Id']


kitti_id = 7373
while(kitti_id < len(lyfttest_Id)):
    
    sample_token = lyfttest_Id[kitti_id]
    """
        prepare test.txt, calibration file and the lidar files
    """
    # write test.txt
    if kitti_id % 10 == 0:
        with open(testtxtfile, 'w') as f:
            print("{:06n}".format(kitti_id),file=f)
    else:
        with open(testtxtfile, 'a') as f:
            print("{:06n}".format(kitti_id),file=f)
    
    # write calibration file
    write_calib(calib_folder,kitti_id)
    
    # extract lidar file path
    lidar_filepath = lidar_filepath_from_id(sample_token)
    # move lidar to path
    lidar_savefilename = "{:06n}".format(kitti_id)+'.bin'
    copyfile(lidar_filepath,os.path.join(lidar_folder, lidar_savefilename)) 
    # write image
    write_image_from_id(image_folder,kitti_id,sample_token)
    

    """
        prediction part
    """
    # predict every 10 files or at the last file
    if kitti_id %10 == 9 or kitti_id ==  len(lyfttest_Id) -1:
        # predition - arges
        with torch.no_grad():

            # create dataloader & network
            test_loader = create_dataloader(logger)
            model = PointRCNN(num_classes=test_loader.dataset.num_class, use_xyz=True, mode='TEST')
            model.cuda()

            load_ckpt_based_on_args(model, logger)

            detect_one_epoch_joint(model, test_loader, epoch_id, root_result_dir, logger, sample_token)
            
            
        
        # remove all the lidar_files and image files
        files = glob.glob(lidar_folder + '/*')
        for f in files:
            os.remove(f)
        files = glob.glob(image_folder + '/*')
        for f in files:
            os.remove(f)
    
        
        
    """
        postprocessing
    """
    
#     #label = label_global_from_rect(sample_token,args)
    
    
   
    
    kitti_id += 1 

2019-11-12 16:50:04,413   INFO  Load testing samples from ../data/KITTI/object/testing
2019-11-12 16:50:04,414   INFO  Done: total test samples 17
2019-11-12 16:50:06,366   INFO  ==> Loading from checkpoint '../trainedModel/car/rcnn/ckpt/rpn30/checkpoint_epoch_2.pth'
2019-11-12 16:50:06,396   INFO  ==> Done
2019-11-12 16:50:06,397   INFO  ---- EPOCH 2 JOINT PREDICTION ----
2019-11-12 16:50:06,397   INFO  ==> Output file: ../predicted/mirror/epoch_2/pred_boxes3d/Car
eval:  20%|██        | 1/5 [00:00<00:02,  1.42it/s, mode=Predicting]

<class 'torch.Tensor'>


eval:  40%|████      | 2/5 [00:01<00:01,  1.68it/s, mode=Predicting]

<class 'torch.Tensor'>


eval:  60%|██████    | 3/5 [00:01<00:01,  1.93it/s, mode=Predicting]

...

<class 'torch.Tensor'>


eval: 100%|██████████| 3/3 [00:01<00:00,  2.60it/s, mode=Predicting]

<class 'torch.Tensor'>
<class 'torch.Tensor'>


eval: 100%|██████████| 3/3 [00:01<00:00,  2.64it/s, mode=Predicting]


KeyboardInterrupt: 